# 从NB到语言模型

## 1. 引言：朴素贝叶斯的局限性

我们知道朴素贝叶斯的局限性来源于其条件独立假设。

它将文本看成是词袋子模型，不考虑词语之间的顺序信息，就会把“武松打死了老虎”与“老虎打死了武松”认作是一个意思。

**那么有没有一种方法提高其对词语顺序的识别能力呢？**

有，就是这里要提到的**N-gram语言模型**。


## 2. N-gram语言模型是啥？

### 2.1从假设性独立到联合概率链规则

照抄我们垃圾邮件识别中的条件独立假设，长这个样子：</p>
<blockquote><p>$P(（“我”,“司”,“可”,“办理”,“正规发票”,“保真”,“增值税”,“发票”,“点数”,“优惠”)|S)$
$=P(“我”|S)×P(“司”|S)×P(“可”|S)×P(“办理”|S)×P(“正规发票”|S)$
$×P(“保真”|S)×P(“增值税”|S)×P(“发票”|S)×P(“点数”|S)×P(“优惠”|S)$</p>
</blockquote>
<p>为了简化起见，我们以字母$x_i$表示每一个词语，并且先不考虑条件“S”。于是上式就变成了下面的独立性公式。</p>
<blockquote><p>$P(x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_{10})$
$=P(x_1)P(x_2)P(x_3)P(x_4)P(x_5)P(x_6)P(x_7)P(x_8)P(x_9)P(x_{10})$
$=P(“我”)P(“司”)P(“可”)P(“办理”)...P(“优惠”)$</p>
</blockquote>
<p>上面的公式要求满足独立性假设，如果去掉独立性假设，我们应该有下面这个<strong>恒等式，即联合概率链规则(chain rule)</strong> ：</p>
<blockquote><p>$P(x_1,x_2,x_3,x_4,x_5,…,x_n)$
$=P(x_1)P(x_2|x_1)P(x_3|x_1,x_2)...P(x_n|x_1,x_2,...,x_{n-1})$</p>
</blockquote>


### 2.2 从联合概率链规则到n-gram语言模型

上面的联合概率链规则公式考虑到词和词之间的依赖关系，但是比较复杂，在实际生活中几乎没办法使用，于是我们就想了很多办法去近似这个公式，比如我们要讲到的语言模型n-gram就是它的一个简化。

如果我们考虑一个词语对上一个词语的依赖关系，公式就简化了如下形式，我们把它叫做二元语法（bigram，2-gram）:

<blockquote><p>$P(x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_{10})$
$=P(x_1)P(x_2|x_1)P(x_3|x_2)P(x_4|x_3)..P(x_{10}|x_9)$
$=P(“我”)P(“司”|“我”)P(“可”|“司”)P(“办理”|“可”)...P(“优惠”|“点数”)$</p>
</blockquote>

如果把依赖词长度再拉长一点，考虑一个词对前两个词的依赖关系，就叫做三元语法（trigram，3-gram），公式如下:

<blockquote><p>$P(x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_{10})$
$=P(x_1)P(x_2|x_1)P(x_3|x_1,x_2)P(x_4|x_2,x_3)×...×P(x_{10}|x_8,x_9)$
$=P(“我”)P(“司”|“我”)P(“可”|“我”,“司”)P(“办理”|“司”,“可”)...P(“优惠”|“发票”,“点数”)$</p>
</blockquote>

如果我们再考虑长一点，考虑n个词语之间的关系，恩恩，这就是n-gram的由来。

其实以上几个简化后的公式，就是著名的**马尔科夫假设(Markov Assumption)**：
下一个词的出现仅依赖于它前面的一个或几个词。这相对于联合概率链规则，其实是一个有点粗糙的简化，不过很好地体现了就近思路，离得较远和关系比较弱的词语就被简化和省略了。实际应用中，这些简化后的n-gram语法比独立性假设还是强很多的。

### 2.3 怎样选择依赖词的个数"n"？

选择依赖词的个数"n"主要与计算条件概率有关。**理论上，只要有足够大的语料，n越大越好，毕竟这样考虑的信息更多。**条件概率很好算，统计一下各个元组出现的次数就可以，比如:

<blockquote><p>$P(“优惠”|“发票”,“点数”)=\frac{(“发票”,“点数”，“优惠”)出现的次数}{(“发票”,“点数”)出现的次数}$</p>
</blockquote>

但我们实际情况往往是**训练语料很有限，很容易产生数据稀疏，不满足大数定律，算出来的概率失真**。

比如(“发票”,“点数”，“优惠”)在训练集中竟没有出现，就会导致零概率问题。

又比如在英文语料库IBM, Brown中，三四百兆的语料，其测试语料14.7%的trigram和2.2%的bigram在训练语料中竟未出现！

另一方面，如果n很大，参数空间过大，也无法实用。假设词表的大小为 100,000，那么n-gram模型的参数数量为 100,000n。

这么多的参数，估计内存就不够放了。

那么，如何选择依赖词的个数n呢？从前人的经验来看：
- 经验上，trigram用的最多。尽管如此，原则上，能用bigram解决，绝不使用trigram。n取≥4的情况较少。
- 更大的n：对下一个词出现的约束信息更多，具有更大的辨别力；
- 更小的n：在训练语料库中出现的次数更多，具有更可靠的统计信息，具有更高的可靠性、实用性。


## 3. N-gram实际应用举例

说了这么N-gram语言模型的背景知识，咱们再来看看N-gram语言模型在自然语言处理中有哪些常见应用。 PS：此部分以原理介绍为多，具体的技术实现细节请参考文中链接或者google。

### 3.1 词性标注

**词性标注是一个典型的多分类问题。**

常见的词性包括名词、动词、形容词、副词等。而一个词可能属于多种词性。如“爱”，可能是动词，可能是形容词，也可能是名词。但是一般来说，“爱”作为动词还是比较常见的。所以统一给“爱”分配为动词准确率也还足够高。这种最简单粗暴的思想非常好实现，如果准确率要求不高则也比较常用。**它只需要基于词性标注语料库做一个统计就够了，连贝叶斯方法、最大似然法都不要用。**词性标注语料库一般是由专业人员搜集好了的，长下面这个样子。其中斜线后面的字母表示一种词性，词性越多说明语料库分得越细：

<img src='./images/corpus.png'/>

需要比较以下各概率的大小，选择概率最大的词性即可：

<blockquote><p>$P(词性_i|“爱")=\frac{“爱"作为“词性_i”的次数}{“爱"出现的次数}  ； i=1,2,3...$</p>
</blockquote>

**但这种方法没有考虑上下文的信息。**

而**一般来说，形容词后面接名词居多，而不接动词，副词后面才接动词，而不接名词。**

考虑到词性会受前面一两个词的词性的影响，可以引入2-gram模型提升匹配的精确度。 

我们匹配以下这句话（已被空格分好词）中“爱”的词性：

<blockquote><p>"闷骚的 李雷 很 爱 韩梅梅"</p>
</blockquote>

将公式进行以下改造，比较各概率的大小，选择概率最大的词性：

<blockquote><p>$P(词性_i|“很”的词性（副词），“爱")=\frac{前面被“副词”修饰的“爱"作为“词性_i”的次数}{前面被“副词”修饰的“爱"出现的次数}  ； i=1,2,3...$</p>
</blockquote>

计算这个概率需要对语料库进行统计。但前提是你得先判断好“很”的词性，因为采用2-gram模型，进而就需要提前判断“李雷”的词性，需要判断“闷骚的”词性。但是“闷骚的”作为第一个词语，已经找不比它更靠前的词语了。这时就可以考虑用之前最简单粗暴的方法判断“闷骚的”的词性，统一判断为形容词即可。

<p>PS:词性标注是自然语言处理中的一项基础性工作，有其细节实现远比我们介绍地更加丰富。感兴趣的同学可以看看这篇文章<a href="https://superangevil.wordpress.com/2009/10/20/nltk5/">《NLTK读书笔记 — 分类与标注》</a></p>

### 3.2 垃圾邮件识别

可以用N-gram进行垃圾邮件识别，而且是朴素贝叶斯方法的进化版。


下面我们用直观的例子探讨一下其在分类问题上是怎么发挥作用的。

一个可行的思路如下：

- 先对邮件文本进行断句，以句尾标点符号（“。” “!” “？”等）为分隔符将邮件内容拆分成不同的句子。
- 用N-gram分类器(马上提到)判断每个句子是否为垃圾邮件中的敏感句子。
- 当被判断为敏感句子的数量超过一定数量（比如3个）的时候，认为整个邮件就是垃圾邮件。

N-gram分类器是什么鬼，这个分类器靠谱么？

**N-gram分类器是结合贝叶斯方法和语言模型的分类器。**
这里用 Y1,Y2分别表示这垃圾邮件和正常邮件，用 X表示被判断的邮件的句子。

根据贝叶斯公式有：

<blockquote><p>$P(Y_i|X)\propto P(X|Y_i)P(Y_i)  ； i=1,2$</p>
</blockquote>

<p>比较i=1和2时两个概率值的大小即可得到$X$所属的分类。对于句子（“我”,“司”,“可”,“办理”,“正规发票”,“保真”,“增值税”,“发票”,“点数”,“优惠”)用字母$X$代表，每一个词语用字母$x_i$表示。$X$就可以写成一个$x_i$组成的向量，$x_i$就是这向量中某个维度的特征。<strong>对$P(X|Y_i)$ 套用2-gram模型。</strong> 则上式化简为：</p>

<blockquote><p>$P(Y_i|X)\propto P(X|Y_i)P(Y_i)  ； i=1,2$
$\propto P(x_1|Y_i)P(x_2|x_1，Y_i)P(x_3|x_2，Y_i)...P(x_{10}|x_9，Y_i)P(Y_i)$
$\propto P(“我”|Y_i)P(“司”|“我”，Y_i)P(“可”|“司”，Y_i)...P(“优惠”|“点数”，Y_i)P(Y_i)$</p>
</blockquote>

公式中的条件概率也比较好求，举个例子：

<blockquote><p>$P(“优惠”|“点数”，Y_i)=\frac{在类别Y_i中，(“点数”，“优惠”)出现的次数}{在类别Y_i中，“点数”出现的次数}$</p>
</blockquote>

剩下的就需要在语料库中间做一个的统计就是了。

**因为这种方法考虑到了词语前面的一个词语的信息，同时也考虑到了部分语序信息，因此区分效果会比单纯用朴素贝叶斯方法更好。**

多提几句，N-gram方法在实际应用中有一些tricks需要注意：
- 3-gram方法的公式与上面类似。此处省略。从区分度来看，3-gram方法更好些。
- 句子开头的词，比如本例中的“我”，因为要考虑其本身作为开头的特征，可以考虑在其前面再添加一个句子起始符号如"<S\>"，这样我们就不必单独计算 P("我"|Yi)，而是替换为计算P("我"|"<S\>",Yi)。形式上与2-gram统一。 这样统计和预测起来都比较方便。
- 一般地，如果**采用N-gram模型，可以在文本开头加入n-1个虚拟的开始符号**，这样在所有情况下预测下一个词的可依赖词数都是一致的。
- 与朴素贝叶斯方法一样，N-gram模型也会发生**零概率问题**，也需要**平滑技术**。 

### 3.3 中文分词

之前说过，中文分词技术是“中文NLP中，最最最重要的技术之一”，重要到某搜索引擎厂有专门的team在集中精力优化这一项工作，重要到能影响双语翻译10%的准确度，能影响某些query下搜索引擎几分之一的广告收入。不过简单的分词实现方式中，包含的原理其实也非常易懂。

说起来，**中文分词也可以理解成一个多分类的问题。**

这里用 X 表示被分词的句子“我司可办理正规发票”， 用 Yi表示该句子的一个分词方案。

咱们继续套用贝叶斯公式：

<blockquote><p>$P(Y_i|X)\propto P(X|Y_i)P(Y_i)  ； i=1,2,3...$</p>
</blockquote>

比较这些概率的大小，找出使得$P(Y_i|X)$ 最大的$Y_i$即可得到$X$ 所属的分类(分词方案)了。

$Y_i$作为分词方案，其实就是个词串，比如（“我司”，“可”，“办理”，“正规发票”）或者（“我”，“司可办”，“理正规”，“发票”），也<strong>就是一个向量</strong>了。

<p>而上面贝叶斯公式中$P(X|Y_i)$ 项的意思就是在分类方案 $Y_i$ 的前提下，其对应句子为$X$ 的概率。而无论分词方案是（“我司”，“可”，“办理”，“正规发票”）还是（“我”，“司可办”，“理正规”，“发票”），或者其他什么方案，其对应的句子都是“我司可办理正规发票”。也就是说<strong>任意假想的一种分词方式之下生成的句子总是唯一的</strong>（只需把分词之间的分界符号扔掉剩下的内容都一样）。于是<strong>可以将 $P(X|Y_i)$ 看作是恒等于1</strong>的。这样贝叶斯公式又进一步化简成为：</p>

<blockquote><p>$P(Y_i|X)\propto P(Y_i)  ； i=1,2,3...$</p>
</blockquote>

<p>也就是说我们只要取最大化的$P(Y_i)$ 就成了。而$Y_i$就是一个词串，也就是一个向量，可以直接套用我们上面的N-gram语言模型。这里采用2-gram。于是有：</p>
<blockquote><p>$P(Y_1)=P(“我司”，“可”，“办理”，“正规发票”)$
$=P(“我司”)P(“可”|“我司”)P(“办理”|“可”)P(“正规发票”|“办理”）$</p>
</blockquote>
<p>第二种分词方案的概率为：</p>
<blockquote><p>$P(Y_2)=P(“我”，“司可办”，“理正规”，“发票”)$
$=P(“我”)P(“司可办”|“我”)P(“理正规”|“司可办”)P(“发票”|“理正规”）$</p>
</blockquote>


<p>由于在语料库中“司可办”与“理正规”一起连续出现的概率为0，于是$P(Y_2)=0$ ,$P(Y_1)$ 的概率更高，优先选择$Y_1$的分词方案。</p>